In [1]:
# Enable horizontal scrolling:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    SparkSession.builder.appName("Batch")  # type: ignore
    .master("spark://spark:7077")
    .config("spark.executor.cores", 1)
    .config("spark.executor.instances", 1)
    .config("spark.cores.max", 2)
    # Set configuration for Notebook display
    .config("spark.sql.repl.eagerEval.enabled", True)
    # Log level to WARN to avoid huge logs
    .config("spark.logConf", False)
    .getOrCreate()
)
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.sparkContext.setLogLevel("WARN")
spark

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /tmp/.ivy/cache
The jars for the packages stored in: /tmp/.ivy/jars
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ebeead9a-49ff-4fe8-9ad4-bb3d0cb1ed0f;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 

In [7]:
from opensea_monitoring.utils.schemas import get_opensea_raw_events_schema

df = spark.read.json(
    "s3a://raw-data/topics/OpenSeaRawEvents/year=*/month=*/day=*/hour=*/*.json.gz",
    schema=get_opensea_raw_events_schema(),
)
# print("Count: ", df.count())
# print("Partitions: ", df.rdd.getNumPartitions())
df

event,topic,payload
item_transferred,collection:*,{item_transferred...
item_transferred,collection:*,{item_transferred...
item_metadata_upd...,collection:*,{item_metadata_up...
item_transferred,collection:*,{item_transferred...
item_transferred,collection:*,{item_transferred...
item_metadata_upd...,collection:*,{item_metadata_up...
item_metadata_upd...,collection:*,{item_metadata_up...
item_transferred,collection:*,{item_transferred...
item_transferred,collection:*,{item_transferred...
item_transferred,collection:*,{item_transferred...


In [8]:
# df.describe().show()
df.printSchema()

root
 |-- event: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- event_type: string (nullable = true)
 |    |-- sent_at: timestamp (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- payload: struct (nullable = true)
 |    |    |-- quantity: integer (nullable = true)
 |    |    |-- listing_date: timestamp (nullable = true)
 |    |    |-- listing_type: string (nullable = true)
 |    |    |-- collection: struct (nullable = true)
 |    |    |    |-- slug: string (nullable = true)
 |    |    |-- item: struct (nullable = true)
 |    |    |    |-- metadata: struct (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- image_url: string (nullable = true)
 |    |    |    |-- permalink: string (nullable = true)
 |    |    |    |-- nft_id: string (nullable = true)
 |    |    |    |-- chain: struct (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 

In [9]:
from pyspark.sql import functions as F
from pyspark.sql import types as T


df_events = df.select(
    "event",
    F.col("payload.event_type").alias("event_type"),
    F.col("payload.payload.collection.slug").alias("collection_slug"),
    F.to_timestamp("payload.sent_at").alias("sent_at"),
    F.col("payload.status").alias("status"),
    F.col("payload.payload.item.metadata.name").alias("item_name"),
    F.col("payload.payload.item.permalink").alias("item_url"),
    F.col("payload.payload.item.nft_id").alias("item_nft_id"),
    F.col("payload.payload.item.metadata.image_url").alias("image_url"),
    F.col("payload.payload.item.chain.name").alias("item_blockchain"),
    F.to_timestamp(F.col("payload.payload.listing_date")).alias("listing_date"),
    F.col("payload.payload.listing_type").alias("listing_type"),
    F.col("payload.payload.from_account.address").alias("from_account"),
    F.col("payload.payload.to_account.address").alias("to_account"),
    F.col("payload.payload.payment_token.symbol").alias("payment_symbol"),
    F.col("payload.payload.payment_token.eth_price").cast(T.DoubleType()).alias("eth_price"),
    F.col("payload.payload.payment_token.usd_price").cast(T.DoubleType()).alias("usd_price"),
    F.col("payload.payload.quantity").cast(T.IntegerType()).alias("quantity"),
).filter(F.col("event") != "phx_reply")
df_events.cache()
df_events.show(truncate=False)

+---------------------+---------------------+--------------------------------------------------+--------------------------+------+-----------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+----------------+------------+------------+------------------------------------------+------------------------------------------+--------------+---------+---------+--------+
|event                |event_type           |collection_slug                                   |sent_at                   |status|item_name                                |item_url                                                                                       

In [10]:
df_events.printSchema()

root
 |-- event: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- collection_slug: string (nullable = true)
 |-- sent_at: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_url: string (nullable = true)
 |-- item_nft_id: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- item_blockchain: string (nullable = true)
 |-- listing_date: timestamp (nullable = true)
 |-- listing_type: string (nullable = true)
 |-- from_account: string (nullable = true)
 |-- to_account: string (nullable = true)
 |-- payment_symbol: string (nullable = true)
 |-- eth_price: double (nullable = true)
 |-- usd_price: double (nullable = true)
 |-- quantity: integer (nullable = true)



# Exploratory Analysis

In [11]:
df_events.select(F.max("sent_at"), F.min("sent_at")).show(truncate=False)

+--------------------------+--------------------------+
|max(sent_at)              |min(sent_at)              |
+--------------------------+--------------------------+
|2024-08-22 23:44:18.107343|2024-08-22 23:29:24.571584|
+--------------------------+--------------------------+



In [12]:
df_events.groupBy("event_type").count().show(truncate=False)

+---------------------+-----+
|event_type           |count|
+---------------------+-----+
|item_transferred     |2423 |
|item_metadata_updated|96   |
+---------------------+-----+



In [21]:
from pyspark.sql.window import Window

transferred_items = df_events.filter(
    (F.col("collection_slug") == "carbonnft-116")
    & (F.col("event_type") == "item_transferred")
)
window_duration = "5 minutes"
slide_duration = None
time_frame_txt = "_".join(window_duration.split())
time_window = F.window("sent_at", window_duration, slide_duration)
collections_assets_transfers = (
    transferred_items.groupBy(
        "collection_slug",
        "item_nft_id",
        "item_name",
        "item_url",
        "image_url",
        time_window,
    )
    .agg(F.count("*").alias("total_transfers"))
    .select(
        F.col("collection_slug").alias("collection"),
        F.col("window.end").alias("timestamp"),
        F.coalesce("item_name", "item_nft_id").alias("asset_name"),
        F.col("item_url").alias("asset_url"),
        "image_url",
        "total_transfers",
    )
)
collections_assets_ranked = collections_assets_transfers.withColumn(
    "rank_by_transfers",
    F.row_number().over(
        Window.partitionBy("collection", "timestamp").orderBy(
            F.desc("total_transfers")
        )
    ),
)
top_collections_assets_events = collections_assets_ranked.select(
    F.lit(f"collection_top_assets_by_transfers__{time_frame_txt}").alias("metric"),
    "timestamp",
    "collection",
    F.col("total_transfers").alias("value"),
    "asset_name",
    "asset_url",
    "image_url",
).filter(F.col("rank_by_transfers") <= 20)
top_collections_assets_events.show(truncate=False)

+---------------------------------------------+-------------------+-------------+-----+--------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------+---------+
|metric                                       |timestamp          |collection   |value|asset_name                                                          |asset_url                                                                                              |image_url|
+---------------------------------------------+-------------------+-------------+-----+--------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------+---------+
|collection_top_assets_by_transfers__5_minutes|2024-08-22 23:35:00|carbonnft-116|1    |avalanche_fuji/0x75093c0484cc1bebbbb048702a127d8d7af365ce/1734659840|https://testnets.opensea.io/ass

In [ ]:
df_events.filter((F.col("usd_price").isNotNull())).groupBy("event_type").count().show(truncate=False)

+-----------+-----+
|event_type |count|
+-----------+-----+
|item_listed|4    |
|item_sold  |2    |
+-----------+-----+



In [ ]:
df_events.filter((F.col("event_type") == "item_sold")).limit(5)

event,event_type,collection_slug,sent_at,status,item_name,item_url,item_nft_id,image_url,item_blockchain,listing_date,listing_type,from_account,to_account,payment_symbol,eth_price,usd_price,quantity
item_sold,item_sold,meta-croak-not-wi...,2024-07-20 15:41:...,NULL,Meta Croak - Not ...,https://testnets....,sepolia/0x50976f5...,https://i.seadn.i...,sepolia,NULL,NULL,NULL,NULL,ETH,1.000000000000000,3505.510000000000...,1
item_sold,item_sold,cryptoverse-contract,2024-07-20 15:54:...,NULL,Blue,https://testnets....,avalanche_fuji/0x...,NULL,avalanche_fuji,NULL,NULL,NULL,NULL,AVAX,0.008079590000000,28.32000000000000...,1


In [ ]:
df_events.filter((F.col("event_type") == "item_listed")).limit(5)

event,event_type,collection_slug,sent_at,status,item_name,item_url,item_nft_id,image_url,item_blockchain,listing_date,listing_type,from_account,to_account,payment_symbol,eth_price,usd_price,quantity
item_listed,item_listed,meta-croak-not-wi...,2024-07-20 15:36:...,NULL,Meta Croak - Not ...,https://testnets....,sepolia/0x50976f5...,https://i.seadn.i...,sepolia,2024-07-20 15:35:57,NULL,NULL,NULL,ETH,1.0,3505.51,1
item_listed,item_listed,cyan-bayc-2,2024-07-20 15:43:...,NULL,NULL,https://testnets....,sepolia/0x300b105...,https://i.seadn.i...,sepolia,2024-07-20 15:43:24,NULL,NULL,NULL,ETH,1.0,3505.51,1
item_listed,item_listed,unidentified-cont...,2024-07-20 19:01:...,NULL,NULL,https://testnets....,avalanche_fuji/0x...,NULL,avalanche_fuji,2024-07-20 19:01:04,NULL,NULL,NULL,AVAX,0.00819324,28.95,1
item_listed,item_listed,testseiberians9,2024-07-21 09:47:...,NULL,Unrevealed Seiber...,https://testnets....,sepolia/0xb37c71e...,https://i.seadn.i...,sepolia,2024-07-21 09:47:41,NULL,NULL,NULL,ETH,1.0,3494.29,1
item_listed,item_listed,unidentified-cont...,2024-07-20 19:15:...,NULL,NULL,https://testnets....,avalanche_fuji/0x...,NULL,avalanche_fuji,2024-07-20 19:15:27,NULL,NULL,NULL,AVAX,0.00819324,28.95,1


In [ ]:
df_events.filter((F.col("event_type") == "item_received_bid")).limit(5)

event,event_type,collection_slug,sent_at,status,item_name,item_url,image_url,item_blockchain,from_account,to_account,payment_symbol,eth_price,usd_price,quantity


In [ ]:
df_events.filter((F.col("event_type") == "item_transferred")).show(truncate=False)

+----------------+----------------+--------------------------------------------------+--------------------------+------+------------------------------+------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+----------------+------------------------------------------+------------------------------------------+--------------+---------+---------+--------+
|event           |event_type      |collection_slug                                   |sent_at                   |status|item_name                     |item_url                                                                                              |image_url                                                                            |item_blockchain |from_account                              |to_account                                |payment_symbol|eth_price|usd_price|quantity|
+----------------+------

# Global Metrics

In [ ]:
time_frame = "1 hour"
time_frame_txt = "_".join(time_frame.split())
time_window = F.window("sent_at", time_frame)

## Marketplace transactions stats

In [ ]:
transferred_items = df_events.filter((F.col("event_type") == "item_transferred"))
transferred_items = transferred_items.withColumn(
    "quantity", F.when(F.col("quantity").cast("int") > 0, F.col("quantity").cast("int")).otherwise(0)
)
transferred_items.printSchema()
transferred_items.show(truncate=False)

root
 |-- event: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- collection_slug: string (nullable = true)
 |-- sent_at: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_url: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- item_blockchain: string (nullable = true)
 |-- from_account: string (nullable = true)
 |-- to_account: string (nullable = true)
 |-- payment_symbol: string (nullable = true)
 |-- eth_price: string (nullable = true)
 |-- usd_price: string (nullable = true)
 |-- quantity: integer (nullable = true)

+----------------+----------------+--------------------------------------------------+--------------------------+------+------------------------------+------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+----------------+-------

In [ ]:
windowed_transactions = (
    transferred_items
    .groupBy(time_window)
    .agg(
        F.count("*").alias("transfers_count"),
        F.sum("quantity").alias("items_transferred_count"),
    )
    .orderBy("window")
    .select(
        F.col("window.start").alias("window_start"),
        F.col("window.end").alias("window_end"),
        "transfers_count",
        "items_transferred_count",
    )
)
windowed_transactions.show(truncate=False)

+-------------------+-------------------+---------------+-----------------------+
|window_start       |window_end         |transfers_count|items_transferred_count|
+-------------------+-------------------+---------------+-----------------------+
|2024-07-20 10:00:00|2024-07-20 11:00:00|4936           |57235                  |
|2024-07-20 11:00:00|2024-07-20 12:00:00|11568          |9184401977             |
+-------------------+-------------------+---------------+-----------------------+



In [ ]:
windowed_transactions_events = windowed_transactions.unpivot(
    ["window_start", "window_end"],
    ["transfers_count", "items_transferred_count"],
    "metric",
    "value"
).select(
    F.concat("metric", F.lit(f"__{time_frame_txt}")).alias("metric"),
    F.col("window_end").alias("timestamp"),
    "value",
    F.lit(None).alias("collection"),
)
windowed_transactions_events.show(truncate=False)

+-------------------------------+-------------------+----------+----------+
|metric                         |timestamp          |value     |collection|
+-------------------------------+-------------------+----------+----------+
|transfers_count__1_hour        |2024-07-20 11:00:00|4936      |NULL      |
|items_transferred_count__1_hour|2024-07-20 11:00:00|57235     |NULL      |
|transfers_count__1_hour        |2024-07-20 12:00:00|11568     |NULL      |
|items_transferred_count__1_hour|2024-07-20 12:00:00|9184401977|NULL      |
+-------------------------------+-------------------+----------+----------+



24/07/20 13:15:20 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-68702f9a-49f7-4b3c-a4a4-14227d12cb09. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-68702f9a-49f7-4b3c-a4a4-14227d12cb09
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimize

## Marketplace Sales Volume over time

In [ ]:
sold_items = df_events.filter((F.col("event_type") == "item_sold"))
# Sales volume over time (Every hour)
sold_items_hourly = (
    sold_items.groupBy(time_window)
    .agg(F.sum("usd_price").alias("usd_volume"), F.count("*").alias("sales_count"))
    .orderBy("window")
    .select(
        F.col("window.start").alias("window_start"),
        F.col("window.end").alias("window_end"),
        "usd_volume",
        "sales_count",
    )
)
sold_items_hourly.show(truncate=False)

+-------------------+-------------------+----------+-----------+
|window_start       |window_end         |usd_volume|sales_count|
+-------------------+-------------------+----------+-----------+
|2024-07-20 11:00:00|2024-07-20 12:00:00|0.534207  |1          |
+-------------------+-------------------+----------+-----------+



In [ ]:
sold_items_hourly_events = sold_items_hourly.select(
    F.lit(f"total_volume__{time_frame_txt}").alias("metric"),
    F.col("window_end").alias("timestamp"),
    F.col("usd_volume").alias("value"),
    F.lit(None).alias("collection"),
)
sold_items_hourly_events

metric,timestamp,value,collection
total_volume__1_hour,2024-07-20 12:00:00,0.534207,NULL


## Top Collections by sales volume

In [ ]:
from pyspark.sql.window import Window

top_collections = (
    sold_items.groupby("collection_slug", time_window)
    .agg(F.sum("usd_price").alias("usd_volume"), F.count("*").alias("sales_count"))
    .orderBy(F.desc("usd_volume"))
    .select(
        "collection_slug",
        F.col("window.start").alias("window_start"),
        F.col("window.end").alias("window_end"),
        "usd_volume",
        "sales_count",
    )
).withColumn(
    "window_rank",
    (
        F.row_number().over(
            Window.partitionBy("window_start", "window_end").orderBy(
                F.desc("usd_volume")
            )
        )
    ),
)
top_collections.show(truncate=False)

+---------------+-------------------+-------------------+----------+-----------+-----------+
|collection_slug|window_start       |window_end         |usd_volume|sales_count|window_rank|
+---------------+-------------------+-------------------+----------+-----------+-----------+
|hhhhhhhh1155   |2024-07-20 11:00:00|2024-07-20 12:00:00|0.534207  |1          |1          |
+---------------+-------------------+-------------------+----------+-----------+-----------+



In [ ]:
top_collections_events = top_collections.select(
    F.lit(f"top_collections_by_volume__{time_frame_txt}").alias("metric"),
    F.col("window_end").alias("timestamp"),
    F.col("usd_volume").alias("value"),
    F.col("collection_slug").alias("collection"),
).filter(F.col("window_rank") <= 10)
top_collections_events.show(truncate=False)

+---------------------------------+-------------------+--------+------------+
|metric                           |timestamp          |value   |collection  |
+---------------------------------+-------------------+--------+------------+
|top_collections_by_volume__1_hour|2024-07-20 12:00:00|0.534207|hhhhhhhh1155|
+---------------------------------+-------------------+--------+------------+



# Collections Metrics

## Top Collections most valuable assets sold

In [ ]:
top_collections_list = (
    top_collections.select("collection_slug")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect()
)

In [ ]:
top_collections_sales = sold_items.filter(
    F.col("collection_slug").isin(top_collections_list)
)
top_collections_assets = (
    top_collections_sales.join(
        top_collections,
        how="left",
        on=(
            (
                top_collections_sales["sent_at"].between(
                    top_collections["window_start"], top_collections["window_end"]
                )
            )
            & (
                top_collections_sales["collection_slug"]
                == top_collections["collection_slug"]
            )
        ),
    )
    .select(
        top_collections_sales["sent_at"],
        top_collections_sales["collection_slug"],
        top_collections_sales["item_name"],
        top_collections_sales["item_url"],
        top_collections_sales["image_url"],
        top_collections_sales["usd_price"],
        top_collections["window_start"],
        top_collections["window_end"],
    )
    .withColumn(
        "rank_by_price",
        F.row_number().over(
            Window.partitionBy("collection_slug", "window_start", "window_end").orderBy(
                F.desc("usd_price")
            )
        ),
    )
)
top_collections_assets

sent_at,collection_slug,item_name,item_url,image_url,usd_price,window_start,window_end,rank_by_price
2024-07-20 11:02:...,hhhhhhhh1155,web3game1155,https://testnets....,https://i.seadn.i...,0.534207000000000000,2024-07-20 11:00:00,2024-07-20 12:00:00,1


In [ ]:
top_collections_assets_events = top_collections_assets.select(
    F.lit(f"collection_top_assets_by_price__{time_frame_txt}").alias("metric"),
    F.col("window_end").alias("timestamp"),
    F.col("collection_slug").alias("collection"),
    F.col("usd_price").alias("value"),
    F.col("item_name").alias("asset_name"),
    F.col("item_url").alias("asset_url"),
    "image_url",
).filter(F.col("rank_by_price") <= 20)
top_collections_assets_events.show(truncate=False)

+--------------------------------------+-------------------+------------+--------------------+------------+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+
|metric                                |timestamp          |collection  |value               |asset_name  |asset_url                                                                           |image_url                                                                            |
+--------------------------------------+-------------------+------------+--------------------+------------+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+
|collection_top_assets_by_price__1_hour|2024-07-20 12:00:00|hhhhhhhh1155|0.534207000000000000|web3game1155|https://testnets.opensea.io/assets/amoy/0xa4ddd7fc0aa185

## Collection Stats

In [ ]:
top_collections_stats = top_collections_assets.groupby(
    "collection_slug",
    "window_start",
    "window_end"
).agg(
    F.min("usd_price").cast("double").alias("floor_price"),
    F.sum("usd_price").cast("double").alias("total_volume"),
    F.avg("usd_price").cast("double").alias("avg_assets_price"),
    F.count("*").cast("double").alias("total_sales"),
)
top_collections_stats

collection_slug,window_start,window_end,floor_price,total_volume,avg_assets_price,total_sales
hhhhhhhh1155,2024-07-20 11:00:00,2024-07-20 12:00:00,0.534207,0.534207,0.534207,1.0


In [ ]:
top_collections_stats_events = top_collections_stats.unpivot(
    ["collection_slug", "window_start", "window_end"],
    ["floor_price", "total_volume", "avg_assets_price", "total_sales"],
    "metric",
    "value",
).select(
    F.concat(F.lit("collection_"), F.col("metric"), F.lit(f"__{time_frame_txt}")).alias("metric"),
    F.col("window_end").alias("timestamp"),
    F.col("collection_slug").alias("collection"),
    "value",
    F.lit(None).alias("asset_name"),
    F.lit(None).alias("asset_url"),
    F.lit(None).alias("image_url"),
)
top_collections_stats_events.show(truncate=False)

+-----------------------------------+-------------------+------------+--------+----------+---------+---------+
|metric                             |timestamp          |collection  |value   |asset_name|asset_url|image_url|
+-----------------------------------+-------------------+------------+--------+----------+---------+---------+
|collection_floor_price__1_hour     |2024-07-20 12:00:00|hhhhhhhh1155|0.534207|NULL      |NULL     |NULL     |
|collection_total_volume__1_hour    |2024-07-20 12:00:00|hhhhhhhh1155|0.534207|NULL      |NULL     |NULL     |
|collection_avg_assets_price__1_hour|2024-07-20 12:00:00|hhhhhhhh1155|0.534207|NULL      |NULL     |NULL     |
|collection_total_sales__1_hour     |2024-07-20 12:00:00|hhhhhhhh1155|1.0     |NULL      |NULL     |NULL     |
+-----------------------------------+-------------------+------------+--------+----------+---------+---------+



## Merge both datasets

In [ ]:
top_collections_stats_events.union(top_collections_assets_events).show(truncate=False)

+--------------------------------------+-------------------+------------+--------------------+------------+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+
|metric                                |timestamp          |collection  |value               |asset_name  |asset_url                                                                           |image_url                                                                            |
+--------------------------------------+-------------------+------------+--------------------+------------+------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+
|collection_floor_price__1_hour        |2024-07-20 12:00:00|hhhhhhhh1155|0.534207            |NULL        |NULL                                                    